In [1]:
from torchvision import transforms
import torch
import numpy as np
import cv2
from matplotlib import pyplot as plt
from PIL import Image
import timeit
import pandas as pd
import gc
from pathlib import Path

import utils.image as im
import utils.cam as cam
import utils.grabcut as gcut
import utils.voc as voc
import utils.json as json
import utils.path as path
from utils.VOCSegmentation import VOCSegmentation

In [2]:
FB_threshold = 0.4

In [3]:
root_path = path.goback_from_current_dir(0)

json_path = root_path + 'json\\'

output_path  = root_path + r'sessions\2 Object grabcut-cam\threshold_0' + str(int(FB_threshold * 10)) + '\\'

In [4]:
Path(output_path).mkdir(parents=True, exist_ok=True)

In [5]:
camnet = cam.Cam()

In [6]:
data_tbl = VOCSegmentation(root = root_path,
                           year = '2012',
                           image_set = 'trainval',
                           download = False,
                           transform = transforms.ToTensor(),
                           target_transform = transforms.ToTensor(),
                           transforms = None,
                           target = 'Object')

In [7]:
data = iter(torch.utils.data.DataLoader(data_tbl,
                                        batch_size = 1,
                                        shuffle = False,
                                        num_workers = 2))

In [8]:
annotations = json.open_json(json_path + "voc-object-annotations-clean")
N = len(annotations)
annotations = iter(annotations.items())

In [ ]:
IoU  = []
Acc  = []
time = []

for i in range(N):
    start = timeit.default_timer()
    gc.collect()
    
    img, sgm = next(data)
    img, sgm = torch.squeeze(img), torch.squeeze(sgm)

    img_pil = img
    img_cv2 = im.pil_to_cv2(img.numpy())
    sgm     = im.f1_to_f255(sgm.numpy())
    
    name, annots = next(annotations)
    
    for j, annot in enumerate(annots):
        c, d, cbbox = annot
        
        img_cam = camnet.get_top(img_pil, c)
        img_cam = cam.cam_process(img_pil, img_cam)
        img_cam = im.bitwise_and(img_cam, im.cbbox_mask(img_cv2, cbbox))
        gcmask_cam = cam.cam_to_gcmask(img_cam, a = 0.0, b = FB_threshold, c = 1.0)
        _, _, pred = gcut.grabcut(img_cv2, gcmask_cam, mode = 'MASK')
        pred    = im.bitwise_and(pred, im.cbbox_mask(img_cv2, cbbox)).astype(bool)
        
        true = voc.true_mask(sgm, j+1).astype(bool)
        
        IoU.append(voc.IoU(true, pred, sgm))
        Acc.append(voc.accuracy(true, pred, sgm))
        
    stop = timeit.default_timer()
    time.append(stop - start)
    if i % 20 == 0:
        print(f'Image nb {i}')
        print(f'Time spent               = ' + voc.time_str(np.sum(time)))
        print(f'Estimated time remaining = ' + voc.time_str(np.mean(time) * (N - 1 - i)))
        print(f'Mean IoU                 =  {np.mean(IoU) :.3f}')
        print(f'Mean Acc                 =  {np.mean(Acc) :.3f}')
        print()

C:\Users\Nicol\Miniconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Image nb 0
Time spent               =   0m00s
Estimated time remaining =  39m57s
Mean IoU                 =  0.195
Mean Acc                 =  0.983

Image nb 20
Time spent               =   0m20s
Estimated time remaining =  47m43s
Mean IoU                 =  0.338
Mean Acc                 =  0.901

Image nb 40
Time spent               =   0m42s
Estimated time remaining =  49m49s
Mean IoU                 =  0.316
Mean Acc                 =  0.919

Image nb 60
Time spent               =   1m09s
Estimated time remaining =  54m24s
Mean IoU                 =  0.301
Mean Acc                 =  0.930

Image nb 80
Time spent               =   1m37s
Estimated time remaining =  56m44s
Mean IoU                 =  0.337
Mean Acc                 =  0.941

Image nb 100
Time spent               =   2m02s
Estimated time remaining =  56m40s
Mean IoU                 =  0.327
Mean Acc                 =  0.941

Image nb 120
Time spent               =   2m32s
Estimated time remaining =  58m45s
Mean IoU   

Image nb 1080
Time spent               =  19m57s
Estimated time remaining =  33m48s
Mean IoU                 =  0.364
Mean Acc                 =  0.937

Image nb 1100
Time spent               =  20m20s
Estimated time remaining =  33m29s
Mean IoU                 =  0.364
Mean Acc                 =  0.938

Image nb 1120
Time spent               =  20m41s
Estimated time remaining =  33m04s
Mean IoU                 =  0.364
Mean Acc                 =  0.937

Image nb 1140
Time spent               =  21m01s
Estimated time remaining =  32m39s
Mean IoU                 =  0.364
Mean Acc                 =  0.937

Image nb 1160
Time spent               =  21m25s
Estimated time remaining =  32m20s
Mean IoU                 =  0.364
Mean Acc                 =  0.937

Image nb 1180
Time spent               =  21m45s
Estimated time remaining =  31m54s
Mean IoU                 =  0.364
Mean Acc                 =  0.936

Image nb 1200
Time spent               =  22m07s
Estimated time remaining =  31m32

Image nb 2160
Time spent               =  39m28s
Estimated time remaining =  13m44s
Mean IoU                 =  0.383
Mean Acc                 =  0.934

Image nb 2180
Time spent               =  39m48s
Estimated time remaining =  13m21s
Mean IoU                 =  0.383
Mean Acc                 =  0.934

Image nb 2200
Time spent               =  40m14s
Estimated time remaining =  13m01s
Mean IoU                 =  0.382
Mean Acc                 =  0.934

Image nb 2220
Time spent               =  40m35s
Estimated time remaining =  12m38s
Mean IoU                 =  0.382
Mean Acc                 =  0.935

Image nb 2240
Time spent               =  40m58s
Estimated time remaining =  12m17s
Mean IoU                 =  0.381
Mean Acc                 =  0.935

Image nb 2260
Time spent               =  41m22s
Estimated time remaining =  11m55s
Mean IoU                 =  0.381
Mean Acc                 =  0.934

Image nb 2280
Time spent               =  41m42s
Estimated time remaining =  11m33

In [ ]:
print(f'Mean IoU = {np.mean(IoU):.3f}')
print(f'Mean Acc = {np.mean(Acc):.3f}')

In [ ]:
df = pd.read_csv(json_path + 'voc-object-annotations-clean.csv')

In [ ]:
df['IoU'] = IoU
df['Accuracy'] = Acc

In [ ]:
df.to_csv(output_path + 'IoU.csv', index = False)